# Práctica minería de texto

* [Presentación](#opcion-1)
* [Técnicas utilizadas](#opcion-2)
    * [Limpieza de texto](#opcion-2-1)
    * [Stemming](#opcion-2-2)
    * [Lematización](#opcion-2-3)
    * [Entidades nombradas(*ENR*)](#opcion-2-4)
* [Solución](#opcion-3)
    * [Extracción de la información](#opcion-3-1)
    * [Tratamiento de los textos](#opcion-3-2)
    * [Agrupación y resultado](#opcion-3-3)
* [Conclusiones](#opcion-4)


### Presentación <a class="anchor" id="opcion-1"></a>

El ejercicio consiste en el tratamiento de una serie de noticias, descargadas de distintos diarios electrónicos, y que deben se agrupadas de forma automática según su contenido.

Para ello hay que hacer uso de distintas técnicas de tratamiento de lenguaje natural: limpieza del texto, stemming, lematizado, tratamiento de entiidades nombradas, etc.

Se parte de un código fuente en el que teniendo disponibles los fichero .txt, se encarga de realizar las agrupación de los textos y compararlo con un array de valores en el que se representa el resultado ideal.
    

### Técnicas utilizadas <a class="anchor" id="opcion-2"></a>

Para la realización del ejercicio se han utilizados las siguientes técnicas de tratamiento de lenguaje natural, haciendo uso de la librería *Python* **NLTK**:

#### - Limpieza del texto: <a class="anchor" id="opcion-2-1"></a> 

En este paso se elimina del texto original los signos de puntuación y aquellas palabras que no aportarán información a los pasos posteriores, denominadas *stopwords*.

In [ ]:
def limpia_signos_puntuacion(texto):
    text_limpio = ''
    for letter in texto:
        if not letter in string.punctuation:
            text_limpio = text_limpio + letter

    return text_limpio

def limpia_stop_words(tokens, idioma):
    if idioma == 'en':
        stopWords = nltk.corpus.stopwords.words('english')
    elif idioma == 'es':
        stopWords = nltk.corpus.stopwords.words('spanish')

    return [w for w in tokens if w.lower() not in stopWords]

#### - Stemming: <a class="anchor" id="opcion-2-2"></a>

Es un método para reducir una palabra a su raíz, haciendo que un tratamiento de búsqueda o agrupación posterior considere dos palabras que tienen esa raíz común como la misma.

In [ ]:
def stemming(tokens, idioma):
    # Seleccionamos el steamer que deseados utilizar.
    if idioma == 'en':
        stemmer = PorterStemmer()
    else:
        stemmer = SnowballStemmer("spanish")

    stemmeds = []

    # Para cada token del texto obtenemos su raíz.
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmeds.append(stemmed)

    return stemmeds

#### - Lematización: <a class="anchor" id="opcion-2-3"></a>

   Método por el cual se simplifica una *forma flexionada* (plural, femenino, conjugada, ...) y sea sustituida por la forma que por norma es aceptada como representación de todas ellas, es decir, la forma que podríamos encontrar en cualquier diccionario.

In [ ]:
def wordnet_value(value):
    result = ''
    # Filtramos las palabras y nos quedamos solo las que nos pueden interesar.
    # Estas son Adjetivos, Verbos, Sustantivos y Adverbios.
    if value.startswith('J'):
        return wordnet.ADJ
    elif value.startswith('V'):
        return wordnet.VERB
    elif value.startswith('N'):
        return wordnet.NOUN
    elif value.startswith('R'):
        return wordnet.ADV
    return result

def lemmatization(tokens, idioma):
    if idioma != 'en':
        return tokens

    wordnet_lemmatizer = WordNetLemmatizer()
    tokens_aux = nltk.pos_tag(tokens)
    lemmatizeds = []

    for token in tokens_aux:
        if len(token) > 0:
            pos = wordnet_value(token[1])
            # Filtramos las palabras que no nos interesan.
            if pos != '':
                lemmatizeds.append(wordnet_lemmatizer.lemmatize(str(token[0]).lower(), pos=pos))

    return lemmatizeds

#### - Entidades nombradas (*ENR*): <a class="anchor" id="opcion-2-4"></a>

   Unidad de información fundamental que se refiere a nombres propios que pueden ser clasificados en categorías variadas.
    
   Las principales categorías son: Personas, Lugares y Organizaciones, si bien pueden aparecer mas segun el tipo de dato (fechas) o el dominio del texto (político, farmacéutico, ...)

In [ ]:
def extrae_entity_names(t):
    entity_names = []
    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extrae_entity_names(child))
    return entity_names

def trata_entity_names(tokens):
    tagged_sentences = [nltk.pos_tag(tokens)]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extrae_entity_names(tree))

    return entity_names

### Solución <a class="anchor" id="opcion-3"></a>

#### - Extración de la información <a class="anchor" id="opcion-3-1"></a>
    
Como primera acción se ha realizado un proceso de extracción de la información desde los fichero *HTML* a *txt*.
Haciendo uso de la librería *BeautifulSoup* se han realizan los siguientes pasos:
- Identificamos el origen. dado que los fichero están descargado y no tenemosla URL, se hace uso de la metainformación guarda en el propio HTML haciendo referencia al origen del mismo:

In [ ]:
         
    origen = bsObj.find(text=lambda text:isinstance(text, Comment))
    if "saved from url" in origen: # puedo identificar desde donde se ha descargado la página
        ....
 

- Según el origen se identifican los *tag's* *html* que contienen tanto el titular de la noticia, para nombrar el *txt* resultante como el cuerpo de la noticia:

In [ ]:

    if "www.theguardian.com" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'class' : 'content__headline'}).text
        objBody = bsObj.find('div', attrs={'itemprop' : 'articleBody'})


en algún caso, además del titular, se extrae una segunda cabecera de la noticia, o se elimina información sobrante que la librería extrae junto con el texto:

In [ ]:

    elif "www.telegraph.co.uk" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'itemprop' : 'headline name'}).text
        objBody = bsObj.find('article', attrs={'itemprop': 'articleBody'})
        cad_inicio = '/* dynamic basic css */'
        cad_fin = 'OBR.extern.researchWidget();'
    elif "elpais.com" in origen:
        hayQueTratar = True
        titulo = bsObj.find('h1', attrs={'itemprop': 'headline'}).text
        subtitulo = bsObj.find('h2', attrs={'itemprop': 'alternativeHeadline'}).text
        objBody = bsObj.find('div', attrs={'itemprop': 'articleBody'})


- con la información recogida se guarda en una nueva carpeta las conversiones a *txt* de cada ficheros:

In [ ]:

    f2 = open(folderDestino + "/" + titulo + ".txt", "w")
    f2.write(titulo + "\n")

    if subtitulo != None:
        subtitulo = subtitulo.replace("\n", "")
        f2.write(subtitulo + "\n")

    if objBody != None:
        for parrafo in objBody.findAll('p'):
            aux = parrafo.text

            if cad_inicio != None and cad_fin != None:

                pos_inicio = aux.find(cad_inicio)
                pos_fin = aux.find(cad_fin) + len(cad_fin)

                if pos_inicio != -1 or pos_fin != -1:
                    aux = aux[:pos_inicio] + aux[pos_fin:]

            f2.write(aux + "\n")


#### - Lectura de los *txt* <a class="anchor" id="opcion-3-2"></a>

Se recoge cada uno de los *txt's* generados en el punto anterior y se cargan en memoria:

In [ ]:

    listing = os.listdir(folder + "/txt")
    for file in listing:

        if file.endswith(".txt"):
            url = folder+"/txt/"+file
            f = open(url,encoding="ANSI");
            raw = f.read()
            f.close()
            t = TextBlob(raw)
            idioma = t.detect_language()
            print("File: ", file," escrito en: ", idioma)
            
            raw_limpio = limpia_signos_puntuacion(raw)
            tokens = nltk.word_tokenize(raw_limpio)
            tokens_limpio = limpia_stop_words(tokens, idioma)


#### - Tratamiento de los textos <a class="anchor" id="opcion-3-3"></a>

Sobre los texto leídos se aplican los distintos métodos y herramientas descritos anteriormente:

In [ ]:

        #text = nltk.Text(stemming(tokens_limpio, idioma))

        #text = nltk.Text(lemmatization(tokens_limpio, idioma))

        #text = nltk.Text(stemming(lemmatization(tokens_limpio, idioma),idioma))

        #text = nltk.Text(trata_entity_names(tokens_limpio))

        #text = nltk.Text(stemming(trata_entity_names(tokens_limpio), idioma))


#### - Agrupación y resultado <a class="anchor" id="opcion-3-2"></a>

Tras haber aplicado cada uno de los métodos, se procesde a realizar la agrupación y la comparación con el array de soluciones optimo, obteniendo el porcentaje de exito del proceso:

In [ ]:

    distanceFunction ="cosine"
    #distanceFunction = "euclidean"
    test = cluster_texts(texts,5,distanceFunction)
    print("test: ", test)
    # Gold Standard
    reference =[0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
    print("reference: ", reference)

    # Evaluation
    print("rand_score: ", adjusted_rand_score(reference,test))


### Conclusiones <a class="anchor" id="opcion-4"></a>

Dejando aparte el proceso de extracción del texto desde el HTML, los pasos dados para conseguir la mejor agrupación han sido:

- realizando una primera ejecución del proceso sin modificar el texto leído, obtenemos el siguiente resultado, siendo este el punto de partida y el que obtendrá la peor puntuación:

In [ ]:
test:       [1  0  1  4  1  1  1  4  4  0  0  0  4  1  3  4  1  1  1  2  1  4]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.151515151515

 - sobre el texto leído desde el *txt* se eliminan los signos de puntuación 

In [ ]:
test:       [1  0  1  3  1  1  1  3  3  0  0  0  3  1  4  3  1  1  1  2  1  3]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.151515151515

y las *stopwords*

In [ ]:
test:       [0  3  0  1  0  0  0  4  2  3  3  3  2  0  0  1  0  0  0  0  0  2]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.209205020921

En este caso la eliminación de los signos de puntuación no influye en el resultado, siendo un cambio más defacilidad para poder aplicar otros métodos que  algo que pueda afectar a un cambio en la agrupación de palabras.
La eliminación de las *stopwords* sí influye debido a que se está eliminando palabras que no aportan información pero sí puede  equivocar al proceso de agrupación.

- aplicamos stemming, sin cambios en el resultado, ya que en este tipo de textos las formas gramaticales usadas no son my variadas y tampoco influyen en lo que realmente es el asunto principal del mismo:

In [ ]:
test:       [0  3  0  1  0  0  0  4  2  3  3  3  2  0  0  1  0  0  0  0  0  2]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.209205020921

- aplicamos lematización, con identico resultado, por el mismo motivo que con el steamming. En este caso sólo se aplica a los documentos en inglés:

In [ ]:
test:       [0  3  0  1  0  0  0  4  2  3  3  3  2  0  0  1  0  0  0  0  0  2]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.209205020921

- probamos a combinar ambas técnicas por si pudiese verse alguna mejora, pero se obtiene el mismo resultado que de forma individual:

In [ ]:
test:       [0  3  0  1  0  0  0  4  2  3  3  3  2  0  0  1  0  0  0  0  0  2]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.209205020921

- haciendo uso de las entidades nombradas si se ve un avance realmente considerable en el resultado. esto es debido a que en este tipo de texto, son precisamente esas pocas lpalabras las que definen el texto completo:

In [ ]:
test:       [3  0  3  3  3  2  2  4  0  0  0  0  0  1  1  1  1  3  2  3  2  0]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.918604651163

- aplicamos stemmin sobre la lista de palabras extraidas al aplicar las entidades nombradas. Sorprendentemente el resultado empeora considerablemente. Esto se debe a que al modificar una lista de palabras tan reducida hace que al proceso de clusterización le custe más identificar el grupo de cada noticia

In [ ]:
test:       [0  4  0  0  0  3  3  0  2  4  4  4  2  1  1  1  1  0  3  0  3  2]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.685714285714

Con las pruebas realizadas hasta el momento, con la aplicación de las ENR se obtiene el mejor resultado, produciéndose únicamente un error.
Todas estas pruebas se han realizado con los docuemnto en dos idiomas: inglés y español, vamos a repetir las pruebas traduciendo los texto en español al inglés, que tiene la herramientas más depuradas para el tratamiento del lenguaje

- la primera pruab sin signos de puntuación u sin *stopwords* parece bastante prometedor:

In [ ]:
test:       [4  1  4  4  0  0  0  2  1  1  1  1  1  3  3  3  3  0  0  0  0  1]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.722117202268

evidentemente el intentar agrupar textos en distintos idiomas hace el proceso más complicado, y una vez unificado el idioma funciona basttante mejor.

- aplicamos stemming, lematización y la combinación de ambos, y de la misma forma que anteriormente, no se obtiene mejora:

In [ ]:
test:       [4  1  4  4  0  0  0  3  1  1  1  1  1  2  2  2  2  0  0  0  0  1]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.722117202268


test:       [4  3  4  4  0  0  0  2  3  3  3  3  3  1  1  1  1  0  0  0  0  3]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.722117202268


test:       [4  1  4  4  0  0  0  3  1  1  1  1  1  2  2  2  2  0  0  0  0  1]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.722117202268

- finalmente realizamos la prueba con ENR, y nuevamente se muestra como el método que mejores resultados da:

In [ ]:
test:       [2  0  2  2  1  4  4  1  0  0  0  0  0  3  3  3  3  1  4  2  4  0]
reference:  [0, 5, 0, 0, 0, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 2, 0, 2, 5]
rand_score:  0.914285714286